<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/Cleaning_Daily_Dialog_Oct18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.7 MB/s eta 0:00:00


In [18]:
from datasets import load_dataset
import re

dataset = load_dataset("HamdanXI/cleaned_daily_dialog_sentence")

def remove_spaces_before_punctuation(example):
    example['dialogue'] = re.sub(r'\s([?.!"](?:\s|$))', r'\1', example['dialogue'])
    return example

def add_space_after_full_stop(example):
    # Add space after full stop if there isn't one already
    example['dialogue'] = re.sub(r'(?<=[.?!])(?=[^\s])', r' ', example['dialogue'])
    return example

def replace_first_person_pronouns(example):
    # Define a pattern for first-person pronouns (case insensitive)
    pattern = r'\b(I|me|my|mine|myself|we|us|our|ours|ourselves)\b'
    example['dialogue'] = re.sub(pattern, "PRO.1", example['dialogue'], flags=re.IGNORECASE)
    return example

def replace_second_person_pronouns(example):
    # Define a pattern for second-person pronouns (case insensitive)
    pattern = r'\b(you|your|yours|yourself)\b'
    example['dialogue'] = re.sub(pattern, "PRO.2", example['dialogue'], flags=re.IGNORECASE)
    return example

def replace_third_person_pronouns(example):
    pattern = r'\b(he|him|his|she|her|hers|it|its|they|them|their|theirs|themselves)\b'
    example['dialogue'] = re.sub(pattern, "PRO.3", example['dialogue'], flags=re.IGNORECASE)
    return example

def replace_question_mark(example):
    example['dialogue'] = re.sub(r'\?', ' QM-W ', example['dialogue'])
    return example

# Apply the transformation
dataset_1 = dataset.map(remove_spaces_before_punctuation)
dataset_2 = dataset_1.map(add_space_after_full_stop)
dataset_3 = dataset_2.map(replace_first_person_pronouns)
dataset_4 = dataset_3.map(replace_second_person_pronouns)
dataset_5 = dataset_4.map(replace_third_person_pronouns)
dataset_6 = dataset_5.map(replace_question_mark)

# To view the changes, you can print out the first few entries
print(dataset_1['train'][:5])
print(dataset_2['train'][:5])
print(dataset_3['train'][:5])
print(dataset_4['train'][:5])
print(dataset_5['train'][:5])
print(dataset_6['train'][:5])

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

{'dialogue': ['Say , Jim , how about going for a few beers after dinner?', 'You know that is tempting but is really not good for our fitness.', 'What do you mean? It will help us to relax.', "Do you really think so? I don't. It will just make us fat and act silly. Remember last time?", "I guess you are right.But what shall we do? I don't feel like sitting at home."]}
{'dialogue': ['Say , Jim , how about going for a few beers after dinner?', 'You know that is tempting but is really not good for our fitness.', 'What do you mean? It will help us to relax.', "Do you really think so? I don't. It will just make us fat and act silly. Remember last time?", "I guess you are right. But what shall we do? I don't feel like sitting at home."]}
{'dialogue': ['Say , Jim , how about going for a few beers after dinner?', 'You know that is tempting but is really not good for PRO.1 fitness.', 'What do you mean? It will help PRO.1 to relax.', "Do you really think so? PRO.1 don't. It will just make PRO.1 f

In [20]:
def merge_datasets(example, idx, dataset_2=dataset_2):
    example['text'] = dataset_2['train'][idx]['dialogue']
    example['gloss'] = example['dialogue']
    return example

merged_dataset = dataset_6.map(merge_datasets, with_indices=True)

# Step 3: Remove the old 'dialogue' column
merged_dataset = merged_dataset.remove_columns(["dialogue"])

# Your dataset should now have the desired structure
print(merged_dataset['train'][:5])

Map:   0%|          | 0/77350 [00:00<?, ? examples/s]

{'text': ['Say , Jim , how about going for a few beers after dinner?', 'You know that is tempting but is really not good for our fitness.', 'What do you mean? It will help us to relax.', "Do you really think so? I don't. It will just make us fat and act silly. Remember last time?", "I guess you are right. But what shall we do? I don't feel like sitting at home."], 'gloss': ['Say , Jim , how about going for a few beers after dinner QM-W ', 'PRO.2 know that is tempting but is really not good for PRO.1 fitness.', 'What do PRO.2 mean QM-W  PRO.3 will help PRO.1 to relax.', "Do PRO.2 really think so QM-W  PRO.1 don't. PRO.3 will just make PRO.1 fat and act silly. Remember last time QM-W ", "PRO.1 guess PRO.2 are right. But what shall PRO.1 do QM-W  PRO.1 don't feel like sitting at home."]}


In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
merged_dataset.push_to_hub("daily_dialog_gloss_FINAL")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/78 [00:00<?, ?ba/s]